# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import re
import nltk 
nltk.download(['punkt','wordnet','averaged_perceptron_tagger'])
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
import pandas as pd
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterRelief_R.db')
df = pd.read_sql_table('DisasterReliefMessage', engine)
X = df['message']
y = df.iloc[:,4:]
y['related'].replace(2, 1, inplace=True)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub(r"[^a-zA-Z0-9]"," ",text.lower())
    tokens=word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens=[]
    for word in tokens:
        clean_word = lemmatizer.lemmatize(word).lower().strip()
        clean_tokens.append(clean_word)
    return clean_tokens
        

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

for i in range(len(y_test.columns)):
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

             precision    recall  f1-score   support

          0       0.64      0.37      0.47      1851
          1       0.83      0.94      0.88      6003

avg / total       0.78      0.80      0.78      7854

             precision    recall  f1-score   support

          0       0.88      0.98      0.93      6476
          1       0.81      0.39      0.52      1378

avg / total       0.87      0.88      0.86      7854

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7821
          1       0.00      0.00      0.00        33

avg / total       0.99      1.00      0.99      7854

             precision    recall  f1-score   support

          0       0.72      0.88      0.79      4600
          1       0.75      0.52      0.62      3254

avg / total       0.74      0.73      0.72      7854

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      7238
          1       0.69      0.07 

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(OneVsRestClassifier(LinearSVC(random_state = 0))))
    ])
parameters = {
              'tfidf__smooth_idf':[True, False],
              'clf__estimator__estimator__C': [1, 2, 5]
             }

cv = GridSearchCV(pipeline, param_grid=parameters, scoring='precision_samples', cv = 3,n_jobs=-1, verbose=2)

cv.fit(X_train, y_train)


Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=True ..........
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=True ..........
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=True ..........
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=False .........
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=False .........
[CV] clf__estimator__estimator__C=1, tfidf__smooth_idf=False .........
[CV] clf__estimator__estimator__C=2, tfidf__smooth_idf=True ..........
[CV] clf__estimator__estimator__C=2, tfidf__smooth_idf=True ..........
[CV] clf__estimator__estimator__C=2, tfidf__smooth_idf=True ..........
[CV] clf__estimator__estimator__C=2, tfidf__smooth_idf=False .........
[CV] clf__estimator__estimator__C=2, tfidf__smooth_idf=False .........
[CV] clf__estimator__estimator__C=2, tfidf__smooth_idf=False .........
[CV] clf__estimator__estimator__C=5, tfidf__smooth_idf=True ..........
[CV] clf__estimat

[Parallel(n_jobs=-1)]: Done   7 out of  18 | elapsed:  6.2min remaining:  9.8min


[CV]  clf__estimator__estimator__C=2, tfidf__smooth_idf=True, total= 4.3min
[CV]  clf__estimator__estimator__C=1, tfidf__smooth_idf=True, total= 4.5min
[CV]  clf__estimator__estimator__C=2, tfidf__smooth_idf=True, total= 4.5min
[CV]  clf__estimator__estimator__C=1, tfidf__smooth_idf=False, total= 5.0min
[CV]  clf__estimator__estimator__C=5, tfidf__smooth_idf=False, total= 5.1min
[CV]  clf__estimator__estimator__C=5, tfidf__smooth_idf=True, total= 5.2min
[CV]  clf__estimator__estimator__C=5, tfidf__smooth_idf=True, total= 5.3min
[CV]  clf__estimator__estimator__C=5, tfidf__smooth_idf=True, total= 5.4min
[CV]  clf__estimator__estimator__C=5, tfidf__smooth_idf=False, total= 1.3min
[CV]  clf__estimator__estimator__C=5, tfidf__smooth_idf=False, total= 1.1min


[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:  7.0min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...nalty='l2', random_state=0, tol=0.0001,
     verbose=0),
          n_jobs=1),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__smooth_idf': [True, False], 'clf__estimator__estimator__C': [1, 2, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='precision_samples', verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
y_pred = cv.predict(X_test)
print("Accuracy")
print((y_pred == y_test).mean())
for i in range(len(y_test.columns)):
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Accuracy
related                   0.827986
request                   0.899287
offer                     0.995798
aid_related               0.765852
medical_help              0.931118
medical_products          0.959766
search_and_rescue         0.972625
security                  0.981156
military                  0.971861
child_alone               1.000000
water                     0.963203
food                      0.944614
shelter                   0.944360
clothing                  0.989305
money                     0.979883
missing_people            0.988159
refugees                  0.971225
death                     0.971734
other_aid                 0.865292
infrastructure_related    0.930354
transport                 0.961548
buildings                 0.961676
electricity               0.979756
tools                     0.994652
hospitals                 0.988923
shops                     0.994652
aid_centers               0.988796
other_infrastructure      0.954800
weather_rel

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(AdaBoostClassifier(GradientBoostingClassifier())))
    ])

parameters = {
#     'vect__ngram_range': ((1, 1), (1, 0)),
    'vect__max_df': (0.5, 0.75, 1.0),
#     'vect__max_features': (None, 5000),
    'tfidf__use_idf': (True, False),
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv = 3 ,n_jobs=-1, verbose=2)
cv.fit(X_train,y_train)
y_pred = cv.predict(X_test)

for i in range(len(y_test.columns)):
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] tfidf__use_idf=True, vect__max_df=0.5 ...........................
[CV] tfidf__use_idf=True, vect__max_df=0.5 ...........................
[CV] tfidf__use_idf=True, vect__max_df=0.5 ...........................
[CV] tfidf__use_idf=True, vect__max_df=0.75 ..........................
[CV] tfidf__use_idf=True, vect__max_df=0.75 ..........................
[CV] tfidf__use_idf=True, vect__max_df=0.75 ..........................
[CV] tfidf__use_idf=True, vect__max_df=1.0 ...........................
[CV] tfidf__use_idf=True, vect__max_df=1.0 ...........................
[CV] tfidf__use_idf=True, vect__max_df=1.0 ...........................
[CV] tfidf__use_idf=False, vect__max_df=0.5 ..........................
[CV] tfidf__use_idf=False, vect__max_df=0.5 ..........................
[CV] tfidf__use_idf=False, vect__max_df=0.5 ..........................
[CV] tfidf__use_idf=False, vect__max_df=0.75 .........................
[CV] tfidf__use_i

### 9. Export your model as a pickle file

In [ ]:
pickle_file_path = 'model.pkl'
pickle.dump(pipeline, open(pickle_file_path, 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.